In [ ]:
# April 2025
# parsers

In [ ]:
from dotenv import load_dotenv
import os

# 1st method: using .env file.
load_dotenv()
# Access them using os.getenv or os.environ
api_key = os.getenv("GROQ_API_KEY")

# 2nd method: using hard code
# api_key = "<put the api key here>"
# if not os.environ.get("GROQ_API_KEY"):
#     os.environ["GROQ_API_KEY"] = api_key #getpass.getpass("Enter API key for Groq: ")




from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
from langchain_core.output_parsers import StrOutputParser


In [ ]:
chain = (
    llm
    | StrOutputParser()
)


In [ ]:
chain.invoke('Tell me about the lunar calander')

In [ ]:
chain = (
    llm
    # | StrOutputParser()
)


In [ ]:
chain.invoke('Tell me about the lunar calander')

In [ ]:
# Without StrOutputParser:
output = llm.invoke("What is the capital of France?")
print(output)  # You have to know how to extract it

# With StrOutputParser:
chain = llm | StrOutputParser()
print('\n\n',chain.invoke("What is the capital of France?"))  # Just gives you '4'

In [ ]:
# JsonOutputParser, however,  is more interesting to me.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser


In [ ]:

chain = llm | JsonOutputParser()

prompt = "Give me a JSON object with your the information about France: { \"country\": \"\", \"capital\": \"\", \"population\": \"\" }"

output = chain.invoke(prompt)

print(output)

In [ ]:
# but it seams like pydanticOutputParser is the one to go for.

In [ ]:
from pydantic import BaseModel, Field

class Country(BaseModel):
    name: str = Field(description="The official name of the country")
    capital: str = Field(description="The capital city")
    population: int = Field(description="Population count")
    currency: str = Field(description="Official currency")
    continent: str = Field(description="Continent where the country is located")


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Initialize the parser
parser = PydanticOutputParser(pydantic_object=Country)

# Generate format instructions
format_instructions = parser.get_format_instructions()

# Create the prompt template
prompt = PromptTemplate(
    template="Provide information about the country '{country_name}' in the following format:\n{format_instructions}",
    input_variables=["country_name"],
    partial_variables={"format_instructions": format_instructions}
)


In [ ]:
# Create the chain by combining the prompt, LLM, and parser
chain = prompt | llm | parser

# Invoke the chain with a specific country
result = chain.invoke({"country_name": "New Zealand"})

# Output the result
print(result)

In [ ]:
# StructuredOutputParser

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
# from langchain.chat_models import ChatOpenAI

# Define response schemas
response_schemas = [
    ResponseSchema(name="name", description="Official name of the country"),
    ResponseSchema(name="capital", description="Capital city"),
    ResponseSchema(name="population", description="Total population")
]

# Initialize the parser with response schemas
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Create a prompt template with format instructions
prompt = PromptTemplate(
    template="Provide information about {country_name}.\n{format_instructions}",
    input_variables=["country_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# # Initialize the language model
# llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Combine the prompt and model
chain = prompt | llm | parser

# Invoke the chain with a specific country
result = chain.invoke({"country_name": "New Zealand"})
print(result)
